In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
#logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
import devlib
from env import TestEnv

from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'aboard',
    
    # Target board IP/MAC address
    "host"        : '192.168.0.1',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'test0000',

}

# Tests Configuration

In [4]:
my_tests_conf = {

    # This is the output folder where all the results will be collected
    "id"      : "ChromeOS_Profiling",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "base",
            "flags"          : "ftrace",
            "sched_features" : "NO_ENERGY_AWARE",
            "cpufreq"        : {
                "governor" : "performance",
            },
        },
        {
            "tag"            : "eas",
            "flags"          : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq"        : {
                "governor" : "performance",
            },
        },
    ],
    
    # Workloads to run (on each platform configuration)
    "wloads" : {
        "perf" : {
            "type" : "perf_bench",
            "conf" : {
                "class" : "messaging",
                "params" : {
                    "group" :   1,
                    "loop"  :   10,
                    "pipe"  : True,
                    "thread": True,
                }
            }
        },
        "rta" : {
            "type" : "rt-app",
            "loadref" : "big",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "p20" : {
                        "kind"   : "periodic",
                        "params" : {
                            "duty_cycle_pct" : 20,
                         },
                    },
                },
            },
        },
    },
    
    # Number of iterations for each workload
    "iterations" : 1,
    
    # FTrace events to collect and functions to profile for all the
    # tests configuration which have the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "cpu_frequency",
         ],
         "buffsize" : 80 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'perf' ],
    
    # Modules required by these experiments
    "modules"     : [ 'bl', 'cpufreq' ],

}

# Tests execution

In [5]:
executor = Executor(my_target_conf, my_tests_conf)

04:04:11  INFO    :         Target - Loading custom (inline) test configuration
04:04:11  INFO    :         Target - Using base path: /home/derkling/Code/schedtest
04:04:11  INFO    :         Target - Loading custom (inline) target configuration
04:04:11  INFO    :         Target - Loading custom (inline) test configuration
04:04:11  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
04:04:11  INFO    :         Target - Connecting linux target with: {'username': 'root', 'host': '192.168.0.1', 'password': 'test0000'}
04:04:16  INFO    :         Target - Initializing target workdir [/root/devlib-target]
04:04:20  INFO    : Target topology: [[0, 1], [2, 3]]
04:04:23  INFO    :       Platform - Loading default EM [/home/derkling/Code/schedtest/libs/utils/platforms/aboard.json]...
04:04:24  INFO    :         FTrace - Enabled events:
04:04:24  INFO    :         FTrace -   ['sched_switch', 'sched_wakeup', 'sched_wakeup_new', 'cpu_frequency']
04:04:24  INFO    :         FTrac

In [6]:
executor.run()

04:04:24  INFO    : 
04:04:24  INFO    : ################################################################################
04:04:24  INFO    :       Executor - Experiments execution
04:04:24  INFO    : ################################################################################
04:04:24  INFO    : 
04:04:24  INFO    : ================================================================================
04:04:24  INFO    :   TargetConfig - configuring target for [base] experiments
04:04:26  INFO    :        CPUFreq - Configuring all CPUs to use [performance] governor
04:04:27  INFO    :          WlGen - Setup new workload rta
04:04:27  INFO    :          RTApp - Workload duration defined by longest task
04:04:27  INFO    :          RTApp - Default policy: SCHED_OTHER
04:04:27  INFO    :          RTApp - ------------------------
04:04:27  INFO    :          RTApp - task [task_p20], sched: using default policy
04:04:27  INFO    :          RTApp -  | calibration CPU: 2
04:04:27  INFO    :   

In [7]:
!tree {executor.te.res_dir}

/home/derkling/Code/schedtest/results/ChromeOS_Profiling
├── perf_bench_messaging:base:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── perf_bench_messaging:eas:perf
│   ├── 1
│   │   ├── output.log
│   │   ├── performance.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:base:rta
│   ├── 1
│   │   ├── output.log
│   │   ├── rta_00.json
│   │   ├── rt-app-task_p20-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:base:single
│   ├── 1
│   │   ├── output.log
│   │   ├── rt-app-task_p20-0.log
│   │   ├── single_00.json
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:eas:rta
│   ├── 1
│   │   ├── output.log
│   │   ├── rta_00.json
│   │   ├── rt-app-task_p20-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.